In [2]:
from tensorflow.keras import backend as K
K.clear_session()

In [3]:
import os
os.chdir("/media/seconddrive/mta_stationing_problem")
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import sys
import datetime as dt
import importlib
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark import SparkConf
import numpy as np
import pickle
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, concatenate, GlobalAveragePooling1D
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
import IPython
from copy import deepcopy
from tqdm import trange, tqdm

from src import tf_utils, config, data_utils, models, linklevel_utils

mpl.rcParams['figure.facecolor'] = 'white'

import warnings

import pandas as pd
import swifter
pd.set_option('display.max_columns', None)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.get_logger().setLevel('INFO')

In [5]:
importlib.reload(tf_utils)
importlib.reload(models)

<module 'src.models' from '/media/seconddrive/mta_stationing_problem/src/models.py'>

In [6]:
import pyspark
print(pyspark.__version__)

3.3.0


In [7]:
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '40g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .getOrCreate()

22/09/09 21:00:28 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/09/09 21:00:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/09 21:00:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
f = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
todelete = apcdata.filter('(load < 0) OR (load IS NULL)').select('transit_date','trip_id','overload_id').distinct()
todelete=todelete.withColumn('marker',F.lit(1))

#joining and whereever the records are not found in sync error table the marker will be null
apcdataafternegdelete=apcdata.join(todelete,on=['trip_id','transit_date','overload_id'],how='left').filter('marker is null').drop('marker')
apcdataafternegdelete = apcdataafternegdelete.sort(['trip_id', 'overload_id'])
apcdataafternegdelete.show(1, vertical=True, truncate=False)
get_columns = ['trip_id', 'transit_date', 'arrival_time', 
               'block_abbr', 'stop_sequence', 'stop_id_original',
               'load', 
               'darksky_temperature', 
               'darksky_humidity', 
               'darksky_precipitation_probability', 
               'route_direction_name', 'route_id',
               'dayofweek',  'year', 'month', 'hour', 'zero_load_at_trip_end',
               'sched_hdwy']
get_str = ", ".join([c for c in get_columns])

apcdataafternegdelete.createOrReplaceTempView("apc")

# # filter subset
query = f"""
SELECT {get_str}
FROM apc
"""
print(query)

apcdataafternegdelete = spark.sql(query)
apcdataafternegdelete = apcdataafternegdelete.na.fill(value=0,subset=["zero_load_at_trip_end"])

In [ ]:
df = apcdataafternegdelete.toPandas()
print(df.shape)
old_shape = df.shape[0]
df.head(1)

In [ ]:
df = df[df.arrival_time.notna()]
df = df[df.sched_hdwy.notna()]
df = df[df.darksky_temperature.notna()]

df['route_id_dir'] = df["route_id"].astype("str") + "_" + df["route_direction_name"]
df['day'] = df["arrival_time"].dt.day
df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

# Adding extra features
# Holidays
fp = os.path.join('data', 'others', 'US Holiday Dates (2004-2021).csv')
holidays_df = pd.read_csv(fp)
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
holidays_df['is_holiday'] = True
df = df.merge(holidays_df[['Date', 'is_holiday']], left_on='transit_date', right_on='Date', how='left')
df['is_holiday'] = df['is_holiday'].fillna(False)
df = df.drop(columns=['Date'])
    
# School breaks
fp = os.path.join('data', 'others', 'School Breaks (2019-2022).pkl')
school_break_df = pd.read_pickle(fp)
school_break_df['is_school_break'] = True
df = df.merge(school_break_df[['Date', 'is_school_break']], left_on='transit_date', right_on='Date', how='left')
df['is_school_break'] = df['is_school_break'].fillna(False)
df = df.drop(columns=['Date'])

# Traffic
# Causes 3M data points to be lost
fp = os.path.join('data', 'traffic', 'triplevel_speed.pickle')
speed_df = pd.read_pickle(fp)
speed_df = speed_df.rename({'route_id_direction':'route_id_dir'}, axis=1)
speed_df = speed_df[['transit_date', 'trip_id', 'route_id_dir', 'traffic_speed']]
df = df.merge(speed_df, how='left', 
                left_on=['transit_date', 'trip_id', 'route_id_dir'], 
                right_on=['transit_date', 'trip_id', 'route_id_dir'])
# df = df[~df['traffic_speed'].isna()]
df['traffic_speed'].bfill(inplace=True)

In [ ]:
old_shape - df.shape[0]

In [ ]:
sorted_df = []
for ba in tqdm(df.block_abbr.unique()):
    ba_df = df[df['block_abbr'] == ba]
    end_stop = ba_df.stop_sequence.max()
    # Same result as creating a fixed_arrival_time (but faster)
    ba_df = ba_df[ba_df.stop_sequence != end_stop].reset_index(drop=True)
    sorted_df.append(ba_df)
        
overall_df = pd.concat(sorted_df)
drop_cols = ['route_direction_name', 'route_id', 'trip_id']
drop_cols = [col for col in drop_cols if col in overall_df.columns]
overall_df = overall_df.drop(drop_cols, axis=1)

# overall_df = overall_df.rename({"fixed_arrival_time": "arrival_time"}, axis=1)

In [ ]:
TIMEWINDOW = 15
overall_df['minute'] = overall_df['arrival_time'].dt.minute
overall_df['minuteByWindow'] = overall_df['minute'] // TIMEWINDOW
overall_df['temp'] = overall_df['minuteByWindow'] + (overall_df['hour'] * 60 / TIMEWINDOW)
overall_df['time_window'] = np.floor(overall_df['temp']).astype('int')
overall_df = overall_df.drop(columns=['minute', 'minuteByWindow', 'temp'])

## Aggregate stops by time window

In [ ]:
print(overall_df.shape)
overall_df.head(1)

In [ ]:
# Group by time windows and get the maximum of the aggregate load/class/sched
# Get mean of temperature (mostly going to be equal)
# TODO: Double check this! 
overall_df = overall_df.groupby(['transit_date', 
                                 'route_id_dir', 
                                 'stop_id_original',
                                 'time_window']).agg({"block_abbr":"first",
                                                      "arrival_time":"first",
                                                      "year":"first", 
                                                      "month":"first",
                                                      "day": "first",
                                                      "hour":"first",
                                                      "is_holiday": "first",
                                                      "is_school_break":"first",
                                                      "dayofweek":"first",
                                                      "zero_load_at_trip_end":"first",
                                                      "stop_sequence":"first",
                                                      "darksky_temperature":"mean", 
                                                      "darksky_humidity":"mean",
                                                      "darksky_precipitation_probability": "mean",
                                                      "traffic_speed":"mean",
                                                      "sched_hdwy": "max",
                                                      "load": "sum" })
overall_df = overall_df.reset_index(level=[0,1,2,3])
overall_df = overall_df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

In [ ]:
print(overall_df.shape)
overall_df[100:120]

In [ ]:
drop_cols = ['arrival_time', 'block_abbr']
drop_cols = [col for col in drop_cols if col in overall_df.columns]
overall_df = overall_df.drop(drop_cols, axis=1)

In [ ]:
# checking bins of loads for possible classification problem
loads = overall_df[overall_df.load <= config.TARGET_MAX]['load']
percentiles = []
for cbin in config.CLASS_BINS:
    percentile = np.percentile(loads.values, cbin)
    percentiles.append(percentile)

# percentiles = [(percentiles[0], percentiles[1]), (percentiles[1] + 1, percentiles[2]), (percentiles[2] + 1, percentiles[3])]
percentiles = [(percentiles[0], percentiles[1]), (percentiles[1] + 1, percentiles[2]), (percentiles[2] + 1, 55.0), (56.0, 75.0), (76.0, 100.0)]
print(f"Percentiles: {percentiles}")
overall_df[config.TARGET_COLUMN_CLASSIFICATION] = overall_df['load'].apply(lambda x: data_utils.get_class(x, percentiles))
overall_df = overall_df[overall_df[config.TARGET_COLUMN_CLASSIFICATION].notna()]
overall_df.y_class.unique()

In [ ]:
overall_df.y_class.value_counts()

In [ ]:
## Hyperparameters
past = 10 # Past stops observed
future = 1 # Future stops predicted
offset = 0

learning_rate = 1e-4
batch_size = 256
epochs = 200

feature_label = config.TARGET_COLUMN_CLASSIFICATION
patience = 10

hyperparams_dict = {'past': past,
                    'future': future,
                    'offset': offset,
                    'learning_rate': learning_rate,
                    'batch_size': batch_size,
                    'epochs': epochs,
                    'patience': patience}
hyperparams_dict

In [ ]:
# target = config.TARGET_COLUMN_CLASSIFICATION
target = 'y_class'

num_columns = ['darksky_temperature', 'darksky_humidity', 'darksky_precipitation_probability', 'sched_hdwy', 'traffic_speed']
cat_columns = ['month', 'hour', 'day', 'stop_sequence', 'stop_id_original', 'year', target]
ohe_columns = ['dayofweek', 'route_id_dir', 'is_holiday', 'is_school_break', 'zero_load_at_trip_end', 'time_window']

columns = num_columns + cat_columns + ohe_columns
print(f"Numerical columns: {num_columns}")
print(f"Categorical columns: {cat_columns}")
print(f"One Hot Encode columns: {ohe_columns}")

In [ ]:
overall_df.head(1)

In [ ]:
overall_df.hour.unique(), overall_df.stop_sequence.unique()

In [ ]:
train_dates = ('2020-01-01', '2021-06-30')
val_dates =   ('2021-06-30', '2021-10-31')
test_dates =  ('2021-10-31', '2022-04-06')

ohe_encoder, label_encoder, num_scaler, train_df, val_df, test_df = linklevel_utils.prepare_linklevel(overall_df, 
                                                                                                 train_dates=train_dates, 
                                                                                                 val_dates=val_dates, 
                                                                                                 test_dates=test_dates,
                                                                                                 cat_columns=cat_columns,
                                                                                                 num_columns=num_columns,
                                                                                                 ohe_columns=ohe_columns,
                                                                                                 feature_label='y_class',
                                                                                                 time_feature_used='transit_date',
                                                                                                 scaler='minmax')

drop_cols = ['transit_date', 'load', 'arrival_time']
drop_cols = [col for col in drop_cols if col in train_df.columns]
train_df = train_df.drop(drop_cols, axis=1)
val_df = val_df.drop(drop_cols, axis=1)
test_df = test_df.drop(drop_cols, axis=1)

arrange_cols = [target] + [col for col in train_df.columns if col != target]
train_df = train_df[arrange_cols]
val_df = val_df[arrange_cols]
test_df = test_df[arrange_cols]

In [ ]:
train_df['y_class'] = train_df.y_class.astype('int')
val_df['y_class']   = val_df.y_class.astype('int')
test_df['y_class']  = test_df.y_class.astype('int')

In [ ]:
print(train_df.shape)
train_df.head(1)

In [ ]:
## Saving encoders, scalers and column arrangement
fp = os.path.join('models', 'same_day', 'LL_OHE_encoder.joblib')
joblib.dump(ohe_encoder, fp)
fp = os.path.join('models', 'same_day', 'LL_Label_encoders.joblib')
joblib.dump(label_encoder, fp)
fp = os.path.join('models', 'same_day', 'LL_Num_scaler.joblib')
joblib.dump(num_scaler, fp)
fp = os.path.join('models', 'same_day', 'LL_X_columns.joblib')
joblib.dump(train_df.columns, fp)

In [ ]:
# Can add shuffle in the future
@tf.autograph.experimental.do_not_convert
def timeseries_dataset_from_dataset(df, feature_slice, label_slice, input_sequence_length, output_sequence_length, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(df.values)
    ds = dataset.window(input_sequence_length + output_sequence_length, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x).batch(input_sequence_length + output_sequence_length)
     
    def split_feature_label(x):
        return x[:input_sequence_length:, feature_slice], x[input_sequence_length:,label_slice]
     
    ds = ds.map(split_feature_label)
     
    return ds.batch(batch_size)


In [ ]:
label_index = train_df.columns.tolist().index(target)
print(label_index)

In [ ]:
label_slice = slice(label_index, label_index + 1, None) # which column the label/labels are
feature_slice = slice(None, None, None) # Which feature columns are included, by default includes all (even label)
input_sequence_length = past # number of past information to look at
output_sequence_length = future # number of time steps to predict

dataset_train = timeseries_dataset_from_dataset(train_df, 
                                                feature_slice=feature_slice,
                                                label_slice=label_slice,
                                                input_sequence_length=input_sequence_length, 
                                                output_sequence_length=output_sequence_length, 
                                                batch_size=batch_size)

dataset_val = timeseries_dataset_from_dataset(val_df, 
                                              feature_slice=feature_slice,
                                              label_slice=label_slice,
                                              input_sequence_length=input_sequence_length, 
                                              output_sequence_length=output_sequence_length, 
                                              batch_size=batch_size)

dataset_test = timeseries_dataset_from_dataset(test_df,
                                               feature_slice=feature_slice,
                                               label_slice=label_slice,
                                               input_sequence_length=input_sequence_length, 
                                               output_sequence_length=output_sequence_length, 
                                               batch_size=batch_size)
for batch in dataset_train.take(1):
    (x, y) = batch
    display(pd.DataFrame(x[100], columns=train_df.columns))
    print(x[100].shape)
    display(pd.DataFrame(y[100], columns=['y_class']))

In [ ]:
num_classes = len(train_df.y_class.unique())
num_classes

In [ ]:
# define model
model = tf.keras.Sequential()
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# compile model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=["sparse_categorical_accuracy"],
)
# model.compile(
#     loss="mean_absolute_error",
#     optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
#     metrics=tf.keras.metrics.MeanSquaredError(),
# )

input_shape = (None, None, len(train_df.columns))
model.build(input_shape)
print(model.summary())

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
# checkpoint_filepath = 'models/same_day/model/CLA_cp-epoch{epoch:02d}-loss{val_loss:.2f}.ckpt'
checkpoint_filepath = 'models/same_day/school_zero_load/CLA_cp-epoch{epoch:02d}-loss{val_loss:.2f}.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
# fit model
callbacks = [keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True), model_checkpoint_callback]

history = model.fit(dataset_train, validation_data=dataset_val, epochs=epochs, callbacks=callbacks, verbose=1)

In [ ]:
# plt.plot(history)

## Inference
* Load model and encoders,scalers,converters

In [ ]:
import datetime as dt

In [ ]:

TIMEWINDOW = 15

In [ ]:
# Load model
latest = tf.train.latest_checkpoint('models/same_day/model')
columns = joblib.load('models/same_day/LL_X_columns.joblib')
label_encoders = joblib.load('models/same_day/LL_Label_encoders.joblib')
ohe_encoder = joblib.load('models/same_day/LL_OHE_encoder.joblib')
num_scaler = joblib.load('models/same_day/LL_Num_scaler.joblib')

In [ ]:
def get_apc_data_for_date(filter_date):
    print("Running this...")
    filepath = os.path.join('data', 'processed', 'apc_weather_gtfs.parquet')
    apcdata = spark.read.load(filepath)
    apcdata.createOrReplaceTempView("apc")

    plot_date = filter_date.strftime('%Y-%m-%d')
    get_columns = ['trip_id', 'transit_date', 'arrival_time', 'vehicle_id', 'ons',
                   'block_abbr', 'stop_sequence', 'stop_name', 'stop_id_original',
                   'load', 
                   'darksky_temperature', 
                   'darksky_humidity', 
                   'darksky_precipitation_probability', 
                   'route_direction_name', 'route_id', 'gtfs_direction_id',
                   'dayofweek',  'year', 'month', 'hour',
                   'sched_hdwy']
    get_str = ", ".join([c for c in get_columns])
    query = f"""
    SELECT {get_str}
    FROM apc
    WHERE (transit_date == '{plot_date}')
    ORDER BY arrival_time
    """
    apcdata = spark.sql(query)
    apcdata = apcdata.withColumn("route_id_dir", F.concat_ws("_", apcdata.route_id, apcdata.route_direction_name))
    apcdata = apcdata.withColumn("day", F.dayofmonth(apcdata.arrival_time))
    apcdata = apcdata.drop("route_direction_name")
    apcdata = apcdata.withColumn("load", F.when(apcdata.load < 0, 0).otherwise(apcdata.load))
    return apcdata

In [ ]:
def prepare_input_data(input_df, ohe_encoder, label_encoders, num_scaler, columns, keep_columns=[], target='y_class'):
    num_columns = ['darksky_temperature', 'darksky_humidity', 'darksky_precipitation_probability', 'sched_hdwy', 'traffic_speed']
    cat_columns = ['month', 'hour', 'day', 'stop_sequence', 'stop_id_original', 'year', 'time_window']
    ohe_columns = ['dayofweek', 'route_id_dir', 'is_holiday']

    # OHE
    input_df[ohe_encoder.get_feature_names_out()] = ohe_encoder.transform(input_df[ohe_columns]).toarray()
    # input_df = input_df.drop(columns=ohe_columns)

    # Label encoder
    for cat in cat_columns:
        encoder = label_encoders[cat]
        input_df[cat] = encoder.transform(input_df[cat])
    
    # Num scaler
    input_df[num_columns] = num_scaler.transform(input_df[num_columns])
    input_df['y_class']  = input_df.y_class.astype('int')

    if keep_columns:
        columns = keep_columns + columns
    # Rearrange columns
    input_df = input_df[columns]
    
    return input_df

def assign_data_to_bins(df, TARGET='load'):
    bins = pd.IntervalIndex.from_tuples([(-1, 6.0), (7.0, 12.0), (13.0, 55.0), (56.0, 75.0), (76.0, 100.0)])
    mycut = pd.cut(df[TARGET].tolist(), bins=bins)
    df['y_class'] = mycut.codes
    return df

In [ ]:
date_to_predict = dt.date(2021, 10, 18)
apcdata = get_apc_data_for_date(date_to_predict)

In [ ]:
df = apcdata.toPandas()
df = df[df.arrival_time.notna()]
df = df[df.sched_hdwy.notna()]
df = df[df.darksky_temperature.notna()]

df['day'] = df["arrival_time"].dt.day
df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

# Adding extra features
# Holidays
fp = os.path.join('data', 'others', 'US Holiday Dates (2004-2021).csv')
holidays_df = pd.read_csv(fp)
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
holidays_df['is_holiday'] = True
df = df.merge(holidays_df[['Date', 'is_holiday']], left_on='transit_date', right_on='Date', how='left')
df['is_holiday'] = df['is_holiday'].fillna(False)
df = df.drop(columns=['Date'])

# Traffic
# Causes 3M data points to be lost
fp = os.path.join('data', 'traffic', 'triplevel_speed.pickle')
speed_df = pd.read_pickle(fp)
speed_df = speed_df.rename({'route_id_direction':'route_id_dir'}, axis=1)
speed_df = speed_df[['transit_date', 'trip_id', 'route_id_dir', 'traffic_speed']]
df = df.merge(speed_df, how='left', 
                left_on=['transit_date', 'trip_id', 'route_id_dir'], 
                right_on=['transit_date', 'trip_id', 'route_id_dir'])
# df = df[~df['traffic_speed'].isna()]
df['traffic_speed'].bfill(inplace=True)

df['minute'] = df['arrival_time'].dt.minute
df['minuteByWindow'] = df['minute'] // TIMEWINDOW
df['temp'] = df['minuteByWindow'] + (df['hour'] * 60 / TIMEWINDOW)
df['time_window'] = np.floor(df['temp']).astype('int')
df = df.drop(columns=['minute', 'minuteByWindow', 'temp'])

# HACK
df = df[df['hour'] != 3]
df = df[df['stop_sequence'] != 0]

df = df.sort_values(by=['block_abbr', 'arrival_time']).reset_index(drop=True)

df = assign_data_to_bins(df, TARGET='load')

In [ ]:
input_df = prepare_input_data(df, ohe_encoder, label_encoders, num_scaler, columns, target='y_class')

In [ ]:
def generate_simple_lstm_predictions(input_df, model, past, future):
    past_df = input_df[0:past]
    future_df = input_df[past:]
    predictions = []
    for f in range(future):
        pred = model.predict(past_df.to_numpy().reshape(1, *past_df.shape))
        y_pred = np.argmax(pred)
        predictions.append(y_pred)
        
        # Add information from future
        last_row = future_df.iloc[[0]]
        last_row['y_class'] = y_pred
        past_df = pd.concat([past_df[1:], last_row])
        
        # Move future to remove used row
        future_df = future_df[1:]
    return predictions

In [ ]:
tdf = input_df[0:20]
model = linklevel_utils.setup_simple_lstm_generator(input_df.shape[1], 5)
model.load_weights(latest)
y_pred = generate_simple_lstm_predictions(tdf, model, 10, 10)

## Setting up grid search config

In [9]:
from itertools import product
past = [1, 3, 5, 7, 9, 11]
time_window = [10, 20, 30, 40, 50, 60]
# layer = [64, 128, 256]
# batch_size = [256, 512]
# learning_rate = [0.1, 0.01, 0.001, 0.0001]
layer = [128]
batch_size = [256]
learning_rate = [0.01]
epochs = [10]
config = [dict(zip(('past',
           'time_window', 
           'layer',
           'batch_size',
           'learning_rate',
           'epochs'), (_past, 
                      _time_window, 
                      _layer,
                      _batch_size,
                      _learning_rate,
                      _epochs))) for _past, _time_window, _layer, _batch_size, _learning_rate, _epochs in product(past, 
                                                                                                                  time_window, 
                                                                                                                  layer,
                                                                                                                  batch_size,
                                                                                                                  learning_rate,
                                                                                                                  epochs)]
len(config)
config[0]

{'past': 1,
 'time_window': 10,
 'layer': 128,
 'batch_size': 256,
 'learning_rate': 0.01,
 'epochs': 10}